<a href="https://colab.research.google.com/github/tyogoutomo/bananayellow/blob/master/instagram-crawler/instagram_crawler_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [0]:
!python3 -m pip install mtcnn
!python3 -m pip install opencv-contrib-python>=4.1.0
!pip install stdiomask
!apt install chromium-chromedriver
!pip install selenium
!pip install cp
!sudo apt-get install python-yaml
!sudo yum install python-yaml
!apt-get update
!apt install python3-dev python3-pip
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [0]:
import os, time, errno, pickle, stdiomask
import re
import getpass
import json
import csv
import yaml
import sys
import cp
import selenium
from selenium import webdriver
from urllib.request import urlopen
from optparse import OptionParser
from selenium import webdriver  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

counter = 0
folllist = []

WINDOW_SIZE = "1360,768"
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
os.chdir("/content/gdrive/My Drive/")

driver = webdriver.Chrome(
    executable_path="/usr/lib/chromium-browser/chromedriver",
    chrome_options=chrome_options
)

# login_id = input('Your IG Account Username: ')
login_id = 'tyogotest'
# password = getpass.getpass('Your IG Account Password: ')
password = 'username'

def login_instagram():
    try:
        driver.get("https://www.instagram.com/accounts/login/")
        time.sleep(2)
        driver.find_element_by_name("username").send_keys(login_id)
        driver.find_element_by_name("password").send_keys(password)
        time.sleep(1)
        driver.find_element_by_class_name("L3NKy").click()
        return login_id
    except:
        print("the Xpath of this element (login_instagram) has changed")
        time.sleep(6)
        driver.close

def followerButton():
    try:
        follButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[2]/a'
        follower_button = driver.find_element_by_xpath(follButtonXpath)
    except ValueError:
        follButtonXpath = '/html/body/div[1]/section/main/div/ul/li[2]/a'
        follower_button = driver.find_element_by_xpath(follButtonXpath)
        pass
    except:
        pass
        print("the Xpath of this element (followerButton) has changed")
    print("button found")
    print(follower_button.text)
    follower_button.click()
    print("button clicked")
    printfoll = (follower_button.text)
    follValue = printfoll.split()
    print(follValue[0])
    follvalue = follValue[0]
    return follvalue

def follGet(n):
    str_n = str(n)
    try:
        follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_n + ']/div/div[1]/div[2]/div[1]/a'
        follower_name = driver.find_element_by_xpath(follXpath)
    except ValueError:
       follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_n + ']/div/div[2]/div[1]/div/div/a'
       follower_name = driver.find_element_by_xpath(follXpath)
    except:
        pass
        print("the Xpath of this element (follGet) has changed")
    time.sleep(0.2)
    follvalue = (follower_name.text)
    return follvalue

def URL():
    max = 100  #maximal userID to take (to avoid account block by Instagram)
    while counter <= max:
        driver.switch_to.window(driver.window_handles[1])
        driver.get("https://www.instagram.com/"+zstr+"/?__a=1")
        time.sleep(0.3)
        userID=driver.find_element_by_xpath('/html/body/pre')
        user=userID.get_attribute('innerHTML')
        s=user
        y=yaml.load(s)
        l=(y.get("logging_page_id"))
        IGid=(l.split('_'))
        IGidVal = IGid[1]
        driver.switch_to.window(driver.window_handles[0])
        if counter >= max:
            print("ID Limit exceded")
            break
        return IGidVal

def myURL():
    driver.switch_to.window(driver.window_handles[1])
    driver.get("https://www.instagram.com/"+login_id+"/?__a=1")
    time.sleep(0.3)
    userID=driver.find_element_by_xpath('/html/body/pre')
    user=userID.get_attribute('innerHTML')
    s=user
    y=yaml.load(s)
    l=(y.get("logging_page_id"))
    IGid=(l.split('_'))
    myIGidVal = IGid[1]
    driver.switch_to.window(driver.window_handles[0])
    return myIGidVal


login_instagram()
time.sleep(5)
driver.get('https://www.instagram.com/tyogo_utomo/')
time.sleep(3)
follvalue=followerButton()
xint = int(follvalue)
time.sleep(3)

#scroll
fBody  = driver.find_element_by_xpath("//div[@class='isgrP']")
scroll = 0
while scroll < xint:
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', fBody)
    time.sleep(0.1)
    scroll += 1
    if scroll == xint/2:
        print("break")
        break
    else:
        pass

driver.execute_script("window.open('');")
myIGidVal = str(myURL())
print(myIGidVal)
time.sleep(1)
for i in range(1, xint):
    #get username
    zstr = str(follGet(i))
    print(str(i)+zstr)
    #get userid
    IDstr = str(URL())
    counter += 1
    if IDstr == "None":
        IDstr = "ID limit exceded"
    else:
        pass
    print(IDstr)
    time.sleep(0.5)
    folllist.append(login_id + "," + myIGidVal + "," + zstr + "," + IDstr)

print(folllist)
time.sleep(3)
with open('followers(' + login_id + ').csv', 'w', newline='') as csvfile:
    wr = csv.writer(csvfile)
    for follower in folllist:
        wr.writerow([follower])
time.sleep(5)
driver.close
driver.close